## Text Preprocessing for Italian Documents

### Import packages

In [ ]:
# %pip install bs4 nltk seaborn wordcloud

In [ ]:
import os
import pickle
import pandas as pd
import re
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Set a clean visual style
sns.set_theme(style="whitegrid", context="notebook")  # or "talk" for presentations



### Load NLTK resources (from the **resources** folder)

This setup is needed because there is a known bug with **italian** resources

In [ ]:
# Add resource folder to path
base_dir = os.getcwd()  # use working dir
resources_path = os.path.join(base_dir, "resources")

# Load italian tokenize
with open(os.path.join(resources_path, "italian_py3.pickle"), 'rb') as f:
    italian_tokenizer = pickle.load(f)

# Load italian stopwords
with open(os.path.join(resources_path, "stopwords_it.txt"), 'r', encoding='utf-8') as f:
    stop_words = set(line.strip() for line in f if line.strip())

### Import dataset from Hugging Face (optional)

In [ ]:
#pip install datasets

# from datasets import load_dataset

# Load the CHANGE-IT dataset from Hugging Face
# dataset = load_dataset("gsarti/change_it", split="train")

# Convert Hugging Face dataset to Pandas DataFrame
# df = dataset.to_pandas()


### Sample **change-it** public dataset (optional)

In [ ]:
# I generate the datasets sampling change-it datasets (you don't need to run this code)

# Load datasets
# df_repubblica = pd.read_csv("change-it/change-it.repubblica.train.csv", sep=',')
# df_ilgiornale = pd.read_csv("change-it/change-it.ilgiornale.train.csv", sep=',')

# Estract a 1% sample
# df_repubblica_sample = df_repubblica.sample(frac=0.01, random_state=42)
# df_ilgiornale_sample = df_ilgiornale.sample(frac=0.01, random_state=42)

# Salva the sample
# df_repubblica_sample.to_csv("data/repubblica_sample.csv", index=False)
# df_ilgiornale_sample.to_csv("data/ilgiornale_sample.csv", index=False)

### Load dataset (stored in **data** folder)

In [ ]:
df = pd.read_csv("data/repubblica_sample.csv")

# add a column with the newspaper name
df['newspaper'] = 'repubblica'

# Print df columns
print(df.columns)

df.head()

### Text cleaning (without stemming)

In [ ]:
import unicodedata

def clean_text_no_stemming(text):
    # 1. Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # 2. Normalize curly quotes and dashes
    text = text.replace("’", "'").replace("‘", "'") \
               .replace("“", '"').replace("”", '"') \
               .replace("–", "-").replace("—", "-")

    # Optional: normalize Unicode characters to ASCII (e.g., é → e)
    # text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")

    # 3. Remove URLs and email addresses
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\S+@\S+', '', text)

    # 4. Lowercase
    text = text.lower()

    # 5. Remove punctuation (keep straight apostrophes and dashes)
    text = re.sub(r"[^\w\s]", '', text)

    # 6. Remove digits
    text = re.sub(r'\d+', '', text)

    # 7. Tokenize and remove stopwords
    sentences = italian_tokenizer.tokenize(text)
    tokens = [word for sent in sentences for word in sent.split()]
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)


### Text cleaning (stemming)

In [ ]:
# Create italian stemmer
stemmer = SnowballStemmer("italian")

def apply_stemming(text):
    tokens = text.split()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)


### Try methods on a sample sentence (playground)

In [ ]:
sample_text = "I giornalisti stavano scrivendo articoli molto interessanti sull'economia italiana."
print("Clean text:", sample_text)

# Fase 1: cleaning
cleaned = clean_text_no_stemming(sample_text)
print("Clean text:", cleaned)

# Fase 2: stemming
stemmed = apply_stemming(cleaned)
print("After stemming:", stemmed)


### Apply text cleaning to the dataset

In [ ]:
# Apply text cleaning
df['cleaned_text'] = df['full_text'].apply(clean_text_no_stemming)
df['stemmed_text'] = df['cleaned_text'].apply(apply_stemming)


### Compute word frequency (using **Counter** method)

In [ ]:
# Join all cleaned texts into a single list of words
all_words = ' '.join(df['cleaned_text']).split()

# Count the frequency of each word
word_freq = Counter(all_words)

In [ ]:
# Show the 10 most frequent words
print("Most frequent words:")
print(word_freq.most_common(10))

# Show the 10 least frequent words
print("\nLeast frequent words:")
print(word_freq.most_common()[-10:])

### 🛠️ Define a function to plot word frequencies

In [ ]:
def plot_top_words(word_freq, title, top_n=10, color='#4C72B0'):
    # Get the top N words and their counts
    words, counts = zip(*word_freq.most_common(top_n))
    
    # Create figure
    plt.figure(figsize=(12, 6))
    bars = sns.barplot(x=list(words), y=list(counts), color=color)

    # Annotate bars with counts
    for i, count in enumerate(counts):
        bars.text(i, count + max(counts)*0.01, str(count), 
                  ha='center', va='bottom', fontsize=10, fontweight='bold')

    # Improve aesthetics
    plt.title(title, fontsize=16, fontweight='bold')
    plt.xlabel("Words", fontsize=12)
    plt.ylabel("Frequency", fontsize=12)
    plt.xticks(rotation=45, fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    plt.show()


### Apply the plot_top_word to before/after word frequencies

In [ ]:
from collections import Counter

# Recalculate frequencies
freq_before = Counter(' '.join(df['full_text']).split())
freq_after = Counter(' '.join(df['cleaned_text']).split())

# Plot comparison
plot_top_words(freq_before, "Top 10 Most Frequent Words (Before Filtering)", color='#1f77b4')
plot_top_words(freq_after, "Top 10 Most Frequent Words (After Filtering)", color='#ff7f0e')


### Define frequent and rare word sets

In [ ]:
# Create sets of most and least frequent words
most_common = set([word for word in word_freq.most_common(10)])
least_common = set([word for word in word_freq.most_common()[-10:]])

print("Words to remove (most frequent):", most_common)
print("Words to remove (least frequent):", least_common)


### 🛠️ Define the filtering function

In [ ]:
# Function to remove both most and least frequent words from a text
def remove_common_rare_words(text):
    tokens = text.split()
    return ' '.join([word for word in tokens if word not in most_common and word not in least_common])


### 🧪 Apply filtering and compare results

In [ ]:
# Apply the filtering function to the cleaned texts
df['final_text'] = df['cleaned_text'].apply(remove_common_rare_words)

# Show comparison between original, cleaned, and final versions
df[['full_text', 'cleaned_text', 'final_text']].head()

### ☁️ Generate word clouds (before and after)

In [ ]:
# Create word cloud from cleaned_text (before filtering)
text_before = ' '.join(df['full_text'])
wordcloud_before = WordCloud(width=800, height=400, background_color='white').generate(text_before)

# Create word cloud from final_text (after filtering)
text_after = ' '.join(df['final_text'])
wordcloud_after = WordCloud(width=800, height=400, background_color='white').generate(text_after)


In [ ]:
# Set up the figure
plt.figure(figsize=(16, 6))

# Word cloud before filtering
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_before, interpolation='bilinear')
plt.title("Word Cloud – Before Filtering", fontsize=14)
plt.axis('off')

# Word cloud after filtering
plt.subplot(1, 2, 2)
plt.imshow(wordcloud_after, interpolation='bilinear')
plt.title("Word Cloud – After Filtering", fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()
